In [1]:
import os
import sys
from scipy.spatial.transform import Rotation as R

%reload_ext autoreload
%autoreload 2

current_directory = os.getcwd()
master_thesis_path = os.path.join(os.path.sep, *current_directory.split(os.path.sep)[:-2])

# Print the master thesis path
print(master_thesis_path)

sys.path.append(os.path.join(master_thesis_path, 'neuralblox'))
sys.path.append(os.path.join(master_thesis_path, 'neuralblox', 'configs'))

#cd to neuralblox folder
os.chdir(os.path.join(master_thesis_path, 'neuralblox'))

import torch
import numpy as np

import open3d as o3d
from src.utils.debug_utils import *
is_cuda = (torch.cuda.is_available())
device = torch.device("cuda" if is_cuda else "cpu")
print(device)
import src.neuralblox.helpers.visualization_utils as vis_utils
import src.neuralblox.helpers.sequential_trainer_utils as st_utils
import src.neuralblox.helpers.metrics_utils as metrics_utils
from src.neuralblox import config_generators

if 'robin' in os.getcwd():
    bool_location = 1
    cfg_path = 'configs/evaluation_cfg.yaml'
    print(f'On home')
elif 'cluster' in os.getcwd():
    bool_location = 2
    print(f'On euler')
else:
    bool_location = 0
    cfg_path = 'configs/evaluation_cfg_local.yaml'
    print(f'On local')

cfg_default_path = 'configs/default.yaml'

cfg = config.load_config(cfg_path, cfg_default_path)

generator_robot = config_generators.get_generator_sequential(cfg, device=device)


/home/robin/Dev/MasterThesis/GithubRepos/master_thesis
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
cuda
On home
/media/robin/T7/report/models/model_backbone.pt
=> Loading checkpoint from local file...
/media/robin/T7/report/models/model_merging.pt
=> Loading checkpoint from local file...


/home/robin/Dev/MasterThesis/GithubRepos/master_thesis/neuralblox/src/checkpoints.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(filename)


Location: home


In [2]:
path_dir = cfg['plotoptix']['path_dir']
og = cfg['plotoptix']['og']
dispatch = cfg['plotoptix']['dispatch']
n_points = cfg['plotoptix']['n_points']
terrains_path = cfg['data']['scene_folder']
out_dir_images = cfg['plotoptix']['out_dir_images']
image_name = cfg['plotoptix']['image_name']

path_data = os.path.join(path_dir, f'data_saving_{n_points}_1_og{og}_dispatch-{dispatch}_.pth')
path_mesh = os.path.join(path_dir, f'mesh_data_saving_{n_points}_1_og{og}_dispatch-{dispatch}_.pth.obj')

data = torch.load(path_data)

terrain = data['model_infos'][0]['category'][0]
transform = data['transform']

angles = transform[0]
rotation = R.from_euler('xyz', angles[0], degrees=True).inv()
translation = transform[1]

query_points = data['query_points']
logits_sampled = data['logits_sampled']
occ_sampled = 1 / (1 + np.exp(-logits_sampled))
occ_sampled = occ_sampled > cfg['plotoptix']['threshold']
query_points_occupied = rotation.apply(query_points[occ_sampled == 1] - translation.numpy())

inputs = data['inputs']
inputs_occ = data['inputs.occ']
inputs_occupied = rotation.apply(inputs[inputs_occ == 1] - translation)

gt_mesh_o3d, gt_mesh_points = metrics_utils.load_ground_truth(cfg, terrain)
gt_mesh_o3d, gt_mesh_points = metrics_utils.apply_transformations(transform, gt_mesh_o3d, gt_mesh_points)
gt_mesh_o3d.translate(-np.array(translation[0])).rotate(rotation.as_matrix(), center=(0, 0, 0))

center_data = np.mean(gt_mesh_o3d.vertices, axis=0)

save_dir = os.path.join(out_dir_images, image_name)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
mesh_path = os.path.join(path_dir, f'mesh_data_saving_{n_points}_1_og{og}_dispatch-{dispatch}_.pth.obj')
mesh_og = o3d.io.read_triangle_mesh(mesh_path)
mesh_og.translate(-np.array([0.5,0.05,0.5])).translate(-np.array(translation[0])).rotate(rotation.as_matrix(), center=(0, 0, 0))
# mesh_og.rotate(rotation.as_matrix(), center=(0, 0, 0)).translate(-np.array([0.5,0,0]))

/tmp/ipykernel_66558/4294805686.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(path_data)


TriangleMesh with 1465813 points and 2931298 triangles.

In [3]:
terrain = data['model_infos'][0]['category'][0]
transform = data['transform']

angles = transform[0]
rotation = R.from_euler('xyz', angles[0], degrees=True).inv()
translation = transform[1]

query_points = data['query_points']
logits_sampled = data['logits_sampled']

occ_sampled = 1 / (1 + np.exp(-logits_sampled))
occ_sampled = occ_sampled > 0.1

query_points_occupied = rotation.apply(query_points[occ_sampled == 1] - translation.numpy())

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(query_points_occupied)
pcd.paint_uniform_color(np.random.rand(3))

inputs = data['inputs']
inputs_occ = data['inputs.occ']
inputs_occupied = rotation.apply(inputs[inputs_occ == 1] - translation)

pcd_inputs = o3d.geometry.PointCloud()
pcd_inputs.points = o3d.utility.Vector3dVector(inputs_occupied)
pcd_inputs.paint_uniform_color(np.random.rand(3))

gt_mesh_o3d, gt_mesh_points = metrics_utils.load_ground_truth(cfg, terrain)
gt_mesh_o3d, gt_mesh_points = metrics_utils.apply_transformations(transform, gt_mesh_o3d, gt_mesh_points)
gt_mesh_o3d.compute_vertex_normals()

gt_mesh_o3d.translate(-np.array(translation[0])).rotate(rotation.as_matrix(), center=(0, 0, 0))

center_data = np.mean(gt_mesh_o3d.vertices, axis=0)

camera_pos = center_data + np.array([0, 8, -20])
camera_target = center_data 
light1_pos = center_data + np.array([-8, 15, 0])
light2_pos = center_data + np.array([8, 15, 0])
light3_pos = center_data + np.array([0, 15, 0])
    
    
camera = o3d.geometry.TriangleMesh.create_cone()
target = o3d.geometry.TriangleMesh.create_cylinder()

target.translate(center_data)

light1 = o3d.geometry.TriangleMesh.create_sphere(radius=0.2)
light2 = o3d.geometry.TriangleMesh.create_sphere(radius=0.2)
light3 = o3d.geometry.TriangleMesh.create_sphere(radius=0.2)
light1.translate(light1_pos)
light2.translate(light2_pos)
light3.translate(light3_pos)

camera.translate(camera_pos)

base_axis = o3d.geometry.TriangleMesh.create_coordinate_frame()
o3d.visualization.draw_geometries([gt_mesh_o3d, pcd, pcd_inputs, camera, target, light1, light2, light3, base_axis])

In [7]:
mesh_path = os.path.join(path_dir, f'mesh_data_saving_{n_points}_1_og{og}_dispatch-{dispatch}_.pth.obj')
mesh_og = o3d.io.read_triangle_mesh(mesh_path)
mesh_og.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_og, pcd])

In [5]:
print(f'Generating meshes')

print(data['inputs'].shape)
print(data['inputs.occ'].shape)
batch_subsampled_reduced = {
    'inputs': data['inputs'].to(torch.float32) + torch.tensor([0.5, +0.09, 0.5]).to(torch.float32),
    'inputs.occ': data['inputs.occ'],
    'points': data['inputs'].to(torch.float32),
    'points.occ': data['inputs.occ']
}
idx_vis = 19

with torch.no_grad():
    mesh_list, inputs_frame_list, times, logits = generator_robot.generate_mesh_at_index(batch_subsampled_reduced, idx_vis, 0, memory_keep=False, generate_logits=True)   


Generating meshes
torch.Size([1, 20, 8192, 3])
torch.Size([1, 20, 8192])


/home/robin/anaconda3/envs/neuralblox_3_8_v2/lib/python3.8/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Generating mesh at index 19
Decoding latent codes from 125 voxels


In [6]:
mesh_o3d = vis_utils.visualize_mesh_and_points(inputs_frame_list, mesh_list)

mesh_path = os.path.join(path_dir, f'mesh_data_saving_{n_points}_1_og{og}_dispatch-{dispatch}_.pth.obj')
#save the mesh 
o3d.io.write_triangle_mesh(mesh_path, mesh_o3d)

[Open3D WARNING] Write OBJ can not include triangle normals.


True

In [3]:
dir_path = '/media/robin/T7/data/pclds/'

sequence_path = os.path.join(dir_path, 'sequence.npy')
sequence_points_path = os.path.join(dir_path, 'sequence_points.npy')
sequence_occupancies_path = os.path.join(dir_path, 'sequence_occupancies.npy')

sequence = torch.tensor(np.load(sequence_path))
sequence_points = torch.tensor(np.load(sequence_points_path))
sequence_occupancies = torch.tensor(np.load(sequence_occupancies_path))

print(sequence.shape)
print(sequence_points.shape)
print(sequence_occupancies.shape)


torch.Size([10000000, 3])
torch.Size([200, 50000, 3])
torch.Size([200, 50000, 1])


In [4]:
generator_robot = config_generators.get_generator_sequential(cfg, device=device)
print(data['inputs'].shape)
print(data['inputs.occ'].shape)
batch_subsampled_reduced = {
    'inputs': sequence_points[::10, :, :].unsqueeze(0).to(torch.float32) ,
    'inputs.occ':sequence_occupancies[::10, :].unsqueeze(0).squeeze(-1).bool(),
    'points': sequence_points[::10, :, :].unsqueeze(0).to(torch.float32),
    'points.occ': sequence_occupancies[::10, :].unsqueeze(0).squeeze(-1).bool()
}
idx_vis = 19

print(batch_subsampled_reduced['inputs'].shape)
print(batch_subsampled_reduced['inputs.occ'].shape)

with torch.no_grad():
    mesh_list, inputs_frame_list, times, logits = generator_robot.generate_mesh_at_index(batch_subsampled_reduced, idx_vis, 0, memory_keep=False, generate_logits=True)   


/media/robin/T7/report/models/model_backbone.pt
=> Loading checkpoint from local file...
/media/robin/T7/report/models/model_merging.pt
=> Loading checkpoint from local file...


/home/robin/Dev/MasterThesis/GithubRepos/master_thesis/neuralblox/src/checkpoints.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(filename)


Location: home
torch.Size([1, 20, 8192, 3])
torch.Size([1, 20, 8192])
torch.Size([1, 20, 50000, 3])
torch.Size([1, 20, 50000])


/home/robin/anaconda3/envs/neuralblox_3_8_v2/lib/python3.8/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Generating mesh at index 19
Decoding latent codes from 101 voxels


In [7]:
mesh_o3d = vis_utils.visualize_mesh_and_points(inputs_frame_list, mesh_list)
mesh_path = os.path.join(path_dir, f'mesh_real_worl_robot.obj')
o3d.io.write_triangle_mesh(mesh_path, mesh_o3d)

[Open3D WARNING] Write OBJ can not include triangle normals.


True

In [6]:
o3d.visualization.draw_geometries([mesh_o3d])